## Text-to-Speech (TTS) Utilities

`add_tts_buttons` attaches a 🗣 button to every prose cell in the dialog. Clicking it reads the cell aloud using the browser's Web Speech API, highlighting each word as it's spoken. Clicking again cancels playback. Key parameters: `rate` controls speed, `pitch` and `volume` adjust voice character, `voice_name` selects a specific browser voice (see `list_voices`), and `hl_color` sets the highlight color.

`list_voices` displays all available browser TTS voices, optionally filtered by a language code prefix (e.g. `'en'`). Google voices are excluded as they lack word-boundary support required for highlighting. Use the output to pick a `voice_name` for `add_tts_buttons`.

In [ ]:
from IPython.display import Javascript, display

In [ ]:
def add_tts_buttons(
    icon='🗣',       # Button emoji
    rate=1,          # Speech speed (0.1-10)
    pitch=1,         # Voice pitch (0-2)
    volume=1,        # Loudness (0-1)
    voice_name='',   # Browser voice name from list_voices()
    hl_color='#ffe066'): # Word highlight color
    "Add TTS buttons to all prose cells"
    display(Javascript(f"""
(function() {{
    var icon = {repr(icon)}, rate = {rate}, pitch = {pitch}, volume = {volume}, voiceName = {'null' if voice_name is None else repr(voice_name)}, hlColor = {repr(hl_color)};
    var emojiRe = /\\p{{Emoji_Presentation}}|\\p{{Extended_Pictographic}}/gu;
    function isEmoji(s) {{ return (emojiRe.lastIndex=0, emojiRe.test(s)); }}
    function getWords(el) {{
        var walker = document.createTreeWalker(el, NodeFilter.SHOW_TEXT, null), words = [];
        while (walker.nextNode()) walker.currentNode.textContent.split(/\\s+/).forEach(function(tok) {{ var t = tok.trim(); if (t && !isEmoji(t)) words.push(t); }});
        return words;
    }}
    function highlightWord(cell, savedHTML, wordIdx) {{
        cell.innerHTML = savedHTML;
        var walker = document.createTreeWalker(cell, NodeFilter.SHOW_TEXT, null), count = 0;
        while (walker.nextNode()) {{
            var node = walker.currentNode, toks = node.textContent.split(/(\\s+)/), offset = 0;
            for (var i = 0; i < toks.length; i++) {{
                var t = toks[i].trim();
                if (t && !isEmoji(t)) {{
                    if (count === wordIdx) {{
                        var span = document.createElement('span');
                        span.style.background = hlColor;
                        span.appendChild(document.createTextNode(toks[i]));
                        var frag = document.createDocumentFragment();
                        if (offset) frag.appendChild(document.createTextNode(node.textContent.slice(0, offset)));
                        frag.appendChild(span);
                        var rest = node.textContent.slice(offset + toks[i].length);
                        if (rest) frag.appendChild(document.createTextNode(rest));
                        node.replaceWith(frag);
                        return;
                    }}
                    count++;
                }}
                offset += toks[i].length;
            }}
        }}
    }}
    function addTTSButtons() {{
        document.querySelectorAll('.prose').forEach(function(cell) {{
            var msgDiv = cell.closest('[id]');
            if (!msgDiv) return;
            var msgId = msgDiv.id;
            var hdr = document.querySelector('#' + msgId + '-o .uk-card-header .flex.justify-end')
                   || document.querySelector('#' + msgId + ' .uk-card-header .flex.justify-end');
            if (!hdr || hdr.querySelector('.tts-btn')) return;
            var btn = document.createElement('div');
            btn.className = 'tts-btn text-muted-foreground uk-btn-ghost hover:text-[hsl(var(--foreground))] transition-all cursor-pointer';
            btn.textContent = icon; btn.style.cssText = 'font-size:1.1em;cursor:pointer;';
            hdr.insertBefore(btn, hdr.firstChild);
            btn.onclick = function() {{
                if (speechSynthesis.speaking) {{ speechSynthesis.cancel(); return; }}
                observer.disconnect();
                var savedHTML = cell.innerHTML;
                var restore = function() {{ cell.innerHTML = savedHTML; observer.observe(document.body, {{childList:true, subtree:true}}); }};
                var words = getWords(cell), wordIdx = 0;
                var utt = new SpeechSynthesisUtterance(words.join(' '));
                utt.rate = rate; utt.pitch = pitch; utt.volume = volume;
                if (voiceName) {{ var v = speechSynthesis.getVoices().find(v => v.name === voiceName); if (v) utt.voice = v; }}
                utt.onboundary = function(e) {{ if (e.name === 'word') highlightWord(cell, savedHTML, wordIdx++); }};
                utt.onend = restore; utt.onerror = restore;
                speechSynthesis.speak(utt);
            }};
        }});
    }}
    var observer = new MutationObserver(addTTSButtons);
    addTTSButtons();
    observer.observe(document.body, {{childList:true, subtree:true}});
}})();
"""))

In [ ]:
def list_voices(lang=None): # Optional language code prefix to filter by (e.g. 'en')
    "Display browser TTS voices with word boundary support"
    filter_js = f"vs = vs.filter(v => v.lang.startsWith({repr(lang)}));" if lang else ""
    display(Javascript(f"""
var out = document.createElement('pre');
document.currentScript.replaceWith(out);
function show(vs) {{
    {filter_js}
    vs = vs.filter(v => !v.name.startsWith('Google'));
    out.textContent = JSON.stringify(vs.map(v => v.name + ' (' + v.lang + ')'), null, 2);
}}
var vs = speechSynthesis.getVoices();
if (vs.length) show(vs);
else speechSynthesis.onvoiceschanged = () => show(speechSynthesis.getVoices());
"""))

In [ ]:
list_voices(lang = 'en')

<IPython.core.display.Javascript object>

In [ ]:
# Karen voice
add_tts_buttons(voice_name = 'Karen', icon='🔊')

I will arise and code now, and go to Solveit,

And a small dialog build there, of cells and kernels made;

Nine code-blocks will I have there, a hive for the busy AI,

And think alone in the prompt-loud glade.

Where will we go now?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

That's up to you! I can see you have some nice utilities set up — TTS buttons, YouTube embedding and transcript tools. A few directions we could go:

- **Read/watch something** — fetch an arXiv paper or YouTube video and discuss it
- **Build something new** — a project, tool, or experiment
- **Learn something** — dive into a topic you're curious about

What are you in the mood for?